Solution that I come up with here does not work for some reason. \
ok it "worked" but I needed to operate on iv and 1st block instead of 1st and 2nd block

In [46]:
# soln
from Crypto.Cipher import AES
import os
from Crypto.Util.Padding import pad, unpad
from datetime import datetime, timedelta
from os import urandom
import requests
from Crypto.Util.number import long_to_bytes, bytes_to_long
import json
def json_get(url):
    return json.loads(requests.get(url).content.decode())
def xor(a, b):
    return long_to_bytes(bytes_to_long(a) ^ bytes_to_long(b))
def check_admin(cookie, iv):
    return json_get(f"https://aes.cryptohack.org/flipping_cookie/check_admin/{cookie}/{iv}")
def get_cookie():
    return bytes.fromhex(json_get("https://aes.cryptohack.org/flipping_cookie/get_cookie/")['cookie'])
cookie = get_cookie()
iv = cookie[:16]
b1 = cookie[16:32]
b2 = cookie[32:]
check_admin(b1.hex(), xor(xor(b"admin=False;expi", b"admin=True;\x05\x05\x05\x05\x05"), iv).hex())

{'flag': 'crypto{4u7h3n71c4710n_15_3553n714l}'}

In [13]:
from Crypto.Cipher import AES
import os
from Crypto.Util.Padding import pad, unpad
from datetime import datetime, timedelta
from os import urandom
from Crypto.Util.number import long_to_bytes, bytes_to_long
def xor(a, b):
    return long_to_bytes(bytes_to_long(a) ^ bytes_to_long(b))
KEY =  urandom(16)
IV = urandom(16)
FLAG = "crypto{aes_flag}"
def bitFlip( pos, bit, data):
    return data[:pos]+bytes([data[pos] ^ bit])+data[pos+1:]


In [2]:
aes = AES.new(KEY, AES.MODE_CBC, IV)
plain = pad(b"admin=False;expiry=10001", 16)
enc = aes.encrypt(plain)
enc

b'6+\n6\xab>=\t\xfb\xff\x9b\xad\xe26\x8d\xbd\x1a\xda\xe8\xeb\xabW\xf4\xf0C\xf8\x19\xf8$\x16\xed\xd4'

In [6]:
pos = 0
ctx = enc
for c in b";admin=True;\x04\x04\x04\x04":
    ctx = bitFlip(pos, plain[pos+16] ^ c, ctx)
    pos+=1
print(AES.new(KEY, AES.MODE_CBC, ctx[:16]).decrypt(ctx[16:]))

b';admin=True;\x04\x04\x04\x04'


In [28]:
out = "77378d108ec9e420801d420d499710944683ae4f8dbcbc5a2c6cb5645c65b5b2c2478c7ddd5a8b5c3c95605aa0eaf81d"
iv = bytes.fromhex(out[:32])
cookie = bytes.fromhex(out[32:])

In [37]:
cookie[:16].hex(), xor(xor(plain[0:16], b"admin=True;\x05\x05\x05\x05\x05"), iv).hex()

('4683ae4f8dbcbc5a2c6cb5645c65b5b2', '77378d108ec9f633990b1c3329ea65f8')

In [11]:
pos = 0
ctx = cookie
for c in b";admin=True;\x04\x04\x04\x04": 
    ctx = bitFlip(pos, plain[pos+16] ^ c, ctx)
    pos+=1
ctx.hex(), ctx[:16].hex()

('fd4f1c01605402232d69532cf98ca19555bba24210e0ebf75c5dd1efe328f96c',
 'fd4f1c01605402232d69532cf98ca195')

In [9]:
from Crypto.Util.py3compat import bchr, bord
padding_len = bord(b";admin=True;\x08\x08\x08\x08"[-1])
bchr(padding_len)*padding_len

b'\x08\x08\x08\x08\x08\x08\x08\x08'